# Introduction to Cross Validation and Model Selection

In a previous lab you created a model with l2 or ridge regularization and l1 or lasso regularization. In both cases, an apparently optimum value of the regularization parameter was found. This process is an example of **model selection**. The goal of model selection is to find the best performing model for the problem at hand. Model selection is a very general term and can apply to at least the following common cases:
- Selection of optimal model **hyperparameters**. Hyperparameters are parameters which determine the characteristics of a model. Hyperparameters are distinct from the model parameters. For example, for the case of l2 regularized regression, the degree of regularization is determined by a hyperparameter, which is distinct from the regression coefficients or parameters. 
- **Feature selection** is the process of determining which features should be used in a model. 
- Comparing different model types is an obvious case of model selection. 

If you are thinking that the model selection process is closely related to model training, you are correct. Model selection is a component of model training. However, one must be careful, as applying a poor model selection method can lead to an over-fit model!

## Overview of k-fold cross validation

The questions remain, how good are the hyperparameter estimates perviously obtained for the l2 and l1 regularization parameters and are there better ways to estimate these parameters? The answer to both questions is to use **resampling methods**. Resampling methods repeat a calculation multiple times using randomly selected subsets of the complete dataset.  In fact, resampling methods are generally the best approach to model selection problems. 


**K-folod Cross validation** is a widely used resampling method. In cross validaton a dataset is divided into **k folds**. Each fold contains $\frac{1}{k}$ cases and is created by **Bernoulli random sampling** of the full data set. A computation is performed on $k-1$ folds of the full dataset. The $k^{th}$ fold is **held back** and is used for testing the result. The compuation is performed $k$ times and model parameters are averaged (mean taken) over the results of the $k$ folds. For each iteration, $k-1$ folds are used for training and the $k^{th}$ fold is used for testing. 

4-fold cross validation is illustrated in the figure below. To ensure the data are randomly sampled the data is randomly shuffled at the start of the procedure. The random samples can then be efficiently sub-sampled as shown in the figure. The model is trained and tested four times. For each iteration the data is trained with three folds of the data and tested with the fold shown in the dark shading. 

<img src="img/CrossValidation.jpg" alt="Drawing" style="width:750px; height:400px"/>
<center> **Resampling scheme for 4-fold cross validation**</center>

## Introduction to nested cross validation

Unfortunately, simple cross validation alone does not provide an unbiased approach to model selection. The problem with evaluating model performance with simple cross validation uses the same data samples as the model selection process. This situation will lead to model over fitting wherein the model selection is learned based on the evaluation data. The result is usually unrealistically optimistic model performance estimates.

To obtain unbiased estimates of expected model performance while performing model selection, it is necessary to use **nested cross validation**. As the name implies, nested cross validation is performed though a pair of nested CV loops. The outer loop uses a set of folds to perform model evaluation. The inner loop performs model selection using another randomly sampled set of  folds not used for evaluation by the outer loop. This algorithm allows model selection and evaluation to proceed with randomly sampled subsets of the full data set, thereby avoiding model selection bias. 

## Cross validation and computational efficiency

As you may have surmised, cross validation can be computationally intensive. Processing each fold of a cross validation requires fitting and evaluating the model. It is desirable  to compute a reasonable number of folds. Since the results are averaged over the folds, a small number of folds can lead to significant variability in the final result. However, with large data sets or complex models, the number of folds must be limited in order to complete the cross validation process in a reasonable amount of time. It is, therefore, necessary to trade off accuracy of the cross validation result with the practical consideration of the required computational resources. 

As mentioned earlier, other resampling methods exist. For example, leave-one-out resampling has the same number of folds as data cases. Such methods provide optimal unbiased estimates of model performance. Unfortunately, as you might think, such methods are computationally intensive and are only suitable for small datasets. In practice k-fold cross validation is a reasonable way to explore bias-variance trade-off with reasonable computational resources. 

## Prepare the data set

With the above theory in mind, you will now try an example. 

As a first step, execute the code in the cell below to load the packages required for this notebook. 

In [1]:
## Import packages
library(ggplot2)
library(repr)
library(dplyr)
library(caret)
library(glmnet)
library(ROCR)

options(repr.plot.width=5, repr.plot.height=5) # Set the initial plot area dimensions


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: lattice
Loading required package: Matrix
Loading required package: foreach
Loaded glmnet 2.0-13

Loading required package: gplots

Attaching package: ‘gplots’

The following object is masked from ‘package:stats’:

    lowess



Next, load the preprocessed files containing the features and the labels. The preprocessing includes the following:
1. Cleaning missing values.
2. Aggregate categories of certain categorical variables. 

Execute the code in the cell below to load the dataset. 

In [2]:
credit = read.csv('German_Credit_Preped.csv')
print(dim(credit))
names(credit)

[1] 1000   22


[1] "Customer_ID"              "checking_account_status" 
 [3] "loan_duration_mo"         "credit_history"          
 [5] "purpose"                  "loan_amount"             
 [7] "savings_account_balance"  "time_employed_yrs"       
 [9] "payment_pcnt_income"      "gender_status"           
[11] "other_signators"          "time_in_residence"       
[13] "property"                 "age_yrs"                 
[15] "other_credit_outstanding" "home_ownership"          
[17] "number_loans"             "job_category"            
[19] "dependents"               "telephone"               
[21] "foreign_worker"           "bad_credit"

### Code the label

The R Caret package computes most performance metrics using the positive cases. For example, recall is a measure of correct classification of positive cases. Therefore, it is important to have the coding of the label correct. The code in the cell below creates a factor (categorical) variable and coerces the levels of the label column, `bad_credit`. Execute this code. 

In [3]:
temp = credit$bad_credit
credit$bad_credit <- ifelse(credit$bad_credit == 1, 'bad', 'good')
credit$bad_credit <- factor(credit$bad_credit, levels = c('bad', 'good'))
credit$bad_credit[1:10]

[1] good bad  good good bad  good good good good bad 
Levels: bad good

### Scale numeric features

Cross validation will be used to train the model. Since folds will be selected from the entire dataset the numeric features are scaled in batch. Execute the code in the cell below to accomplish this: 

In [4]:
num_cols = c('loan_duration_mo', 'loan_amount', 'payment_pcnt_income',
             'time_in_residence', 'age_yrs', 'number_loans', 'dependents')

preProcValues <- preProcess(credit[,num_cols], method = c("center", "scale"))
credit[,num_cols] = predict(preProcValues, credit[,num_cols])
head(credit[,num_cols])

loan_duration_mo,loan_amount,payment_pcnt_income,time_in_residence,age_yrs,number_loans,dependents
-1.2358595,-0.7447588,0.91801781,1.0464631,2.76507291,1.0265652,-0.4280754
2.2470700,0.9493418,-0.86974813,-0.7655942,-1.19080809,-0.7045734,-0.4280754
-0.7382981,-0.4163541,-0.86974813,0.1404344,1.18272051,-0.7045734,2.3337012
1.7495086,1.6334296,-0.86974813,1.0464631,0.83108664,-0.7045734,2.3337012
0.2568246,0.5663801,0.02413484,1.0464631,1.53435438,1.0265652,2.3337012
1.2519473,2.0489838,-0.86974813,1.0464631,-0.04799802,-0.7045734,2.3337012


## Optimize hyperparameters with nested cross validation

Given often observed variability in metrics in cross validation, performing model selection from a single training and evaluation is an uncertain proposition at best. Fortunately, the nested cross validation approach provides a better way to perform model selection. However, there is no guarantee that a model selection process will, in fact, improve a model. In some cases, it may be that model selection has minimal impact. 

The inner cross validation loop is used to find the optimal hyperparameters. The general process is:
1. A grid of hyperparameters is defined. The model selection will be performed by comparing performance metrics over this grid. 
2. The folds for the cross validation are defined.
3. For each set of folds models are computed for each combination of hyperparameters from the grid. 
4. The Average metrics of each model (hyperparameter combination) is computed. The best model is selected based on these averages.

Once the inner cross validation determines optimal hyperparameters, a final cross validation is performed to determine how well the final model is expected to perform. This process is know as the inner loop of the nested CV. Notice that by creating these independent fold objects there is no need to actually create nested loops for this process. While conceptually nesting the inner and out loops is not hard, is computationally intensive and is generally avoided.  

### Inner loop

The code in the cell below uses the capabilities of the Caret package to perform the inner loop of the cross validation as follows:
1. The 'trainControl' object defines a 10 fold cross validation. The `twoClassSummary` function defines the ROC metric as th one used for model selection. 
2. The `train` function defines the following:
  - The model is defined with the R modeling language.
  - The data frame to be used specified.
  - The `method` argument specifies the R model to use.
  - Optional arguments, such as `weights` for the model are defined. 
  - The metric to be used for model evaluation is specified. 
  - The `trainControl` object is specified.

Execute this code and examine the results:

In [5]:
## Create a weight vector for the training cases.
weights = ifelse(credit$bad_credit == 'bad', 0.66, 0.34)

fitControl = trainControl(method = 'cv',
                          number = 10,
                          classProbs = TRUE,
                          summaryFunction = twoClassSummary)

set.seed(9999)
cv_mod_roc = train(bad_credit ~ loan_duration_mo + loan_amount +  
                                payment_pcnt_income + age_yrs + 
                                checking_account_status + credit_history + 
                                purpose + gender_status + time_in_residence +
                                property,
                   data = credit, 
                   method = "glmnet", 
                   weights = weights, 
                   metric="ROC",
                   trControl = fitControl)
    
cv_mod_roc

glmnet 

1000 samples
  10 predictor
   2 classes: 'bad', 'good' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 900, 900, 900, 900, 900, 900, ... 
Resampling results across tuning parameters:

  alpha  lambda        ROC        Sens       Spec     
  0.10   0.0002955172  0.7759048  0.6333333  0.7614286
  0.10   0.0029551720  0.7757619  0.6266667  0.7628571
  0.10   0.0295517205  0.7770000  0.6400000  0.7614286
  0.55   0.0002955172  0.7756667  0.6333333  0.7614286
  0.55   0.0029551720  0.7761905  0.6233333  0.7642857
  0.55   0.0295517205  0.7724762  0.6500000  0.7357143
  1.00   0.0002955172  0.7757619  0.6333333  0.7614286
  1.00   0.0029551720  0.7753810  0.6400000  0.7628571
  1.00   0.0295517205  0.7592857  0.6733333  0.7142857

ROC was used to select the optimal model using the largest value.
The final values used for the model were alpha = 0.1 and lambda = 0.02955172.

The grid of hyperpameters searched by the Caret package is over `alpha` and `lambda`. The printed tables shows the values of the metrics as a function of the parameters in the search grid. Sens is short for sensitivity which is the same as global recall and Spec is specificity which is the true negative rate $= \frac{TN}{TN + FP}$

As an alternative, recall can be used as the model selection metric. The code in the cell below uses the `prSummary` function in the `trainControl` object and the `Recall` as the `metric`. Execute this code and examine the results.

In [6]:
## Create a weight vector for the training cases.
weights = ifelse(credit$bad_credit == 'bad', 0.66, 0.34)

fitControl = trainControl(method = 'cv',
                          number = 10,
                          classProbs = TRUE,
                          summaryFunction = prSummary)

set.seed(9999)
cv_mod_recall = train(bad_credit ~ loan_duration_mo + loan_amount +  
                                   payment_pcnt_income + age_yrs + 
                                   checking_account_status + credit_history + 
                                   purpose + gender_status + time_in_residence +
                                   property,
                      data = credit, 
                      method = "glmnet", 
                      weights = weights, 
                      metric="Recall",
                      trControl = fitControl)
    
cv_mod_recall

glmnet 

1000 samples
  10 predictor
   2 classes: 'bad', 'good' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 900, 900, 900, 900, 900, 900, ... 
Resampling results across tuning parameters:

  alpha  lambda        AUC        Precision  Recall     F        
  0.10   0.0002955172  0.5773434  0.5331288  0.6333333  0.5773607
  0.10   0.0029551720  0.5758529  0.5323636  0.6266667  0.5740510
  0.10   0.0295517205  0.5654062  0.5355771  0.6400000  0.5816481
  0.55   0.0002955172  0.5766628  0.5331288  0.6333333  0.5773607
  0.55   0.0029551720  0.5754488  0.5330042  0.6233333  0.5728910
  0.55   0.0295517205  0.5547080  0.5169157  0.6500000  0.5736465
  1.00   0.0002955172  0.5767051  0.5331288  0.6333333  0.5773607
  1.00   0.0029551720  0.5737468  0.5393625  0.6400000  0.5834460
  1.00   0.0295517205  0.5276405  0.5059268  0.6733333  0.5760049

Recall was used to select the optimal model using the largest value.
The final values used for the model were

The results of the second CV are nearly the same as the first one.

You can illustrate the performance of the different models over the hyperparameter grid. Execute the code in the cell below to create this display. 

In [7]:
plot(cv_mod_roc, metric = "ROC", plotType = "level",
     scales = list(x = list(rot = 90)))

You can see that the best model in the upper left. 

### Outer loop

With the model selected, you will now perform the outer loop of the cross validation to verify the performance of the model. Consistent performance across the folds indicates that the model is likely to generalize well when faced with new data values. 

The code in the cell below executes the outside CV loop:
1. The parameter grid is specified to have only the optimal hyperparameters. 
2. The `trainControl` object uses the `savePredictions` and `returnResamp` arguments to save the results of each of the folds. 
3. The `tuneGrid` argument of the `train` function is used to specify the parameter grid. 

Execute this code, examine the results, and answer **Question 1** on the course page.

In [8]:
## Set the hyperparameter grid to the optimal values from the inside loop
paramGrid <- expand.grid(alpha = c(cv_mod_roc$bestTune$alpha),
                         lambda = c(cv_mod_roc$bestTune$lambda))

fitControl = trainControl(method = 'cv',
                          number = 10,
                          returnResamp="all",
                          savePredictions = TRUE,
                          classProbs = TRUE,
                          summaryFunction = twoClassSummary)

set.seed(1234)
cv_mod_outer = train(bad_credit ~ loan_duration_mo + loan_amount +  
                                  payment_pcnt_income + age_yrs + 
                                  checking_account_status + credit_history + 
                                  purpose + gender_status + time_in_residence +
                                  property,
                     data = credit, 
                     method = "glmnet", 
                     weights = weights, 
                     tuneGrid = paramGrid, 
                     metric="ROC",
                     trControl = fitControl)
    
print_metrics = function(mod){
    means = c(apply(mod$resample[,1:3], 2, mean), alpha = mod$resample[1,4], 
                lambda = mod$resample[1,5], Resample = 'Mean')
    stds = c(apply(mod$resample[,1:3], 2, sd), alpha = mod$resample[1,4], 
                lambda = mod$resample[1,5], Resample = 'STD')
    out = rbind(mod$resample, means, stds)
    out[,1:3] = lapply(out[,1:3], function(x) round(as.numeric(x), 3))
    out
}
print_metrics(cv_mod_outer)

ROC,Sens,Spec,alpha,lambda,Resample
0.791,0.700,0.671,0.1,0.029551720478006,Fold01
0.716,0.700,0.786,0.1,0.029551720478006,Fold02
0.825,0.633,0.857,0.1,0.029551720478006,Fold03
0.722,0.533,0.671,0.1,0.029551720478006,Fold04
0.797,0.700,0.714,0.1,0.029551720478006,Fold05
0.774,0.633,0.786,0.1,0.029551720478006,Fold06
0.851,0.833,0.800,0.1,0.029551720478006,Fold07
0.809,0.633,0.786,0.1,0.029551720478006,Fold08
0.806,0.600,0.771,0.1,0.029551720478006,Fold09
0.638,0.500,0.686,0.1,0.029551720478006,Fold10


Notice the following about these results: 
1. There is variation in the three performance metrics from fold to fold. This variation gives you an idea of the variability to be expected when the model is placed in production.
2. The mean of each metric summarizes the expected performance of the model. 
3. The standard deviation of each metric summarizes the variability. You can see that the standard deviation is approximately an order of magnitude less than the metric. 

In summary, these results indicate this model is likely to generalize well since the variation in the performance metrics is limited. 

## Summary

In this lab you have performed by simple cross validation and nested cross validation. Key points and observations are:
1. Model selection should be done using a resampling procedure such as nested cross validation. The nested sampling structure is required to prevent bias in model selection wherein the model selected learns the best hyperparameters for the samples used, rather than a model that generalizes well. 
2. There is significant variation in model performance from fold to fold in cross validation. This variation arises from the sampling of the data alone and is not a property of any particular model.
3. Given the expected sampling variation in cross validation, there is generally considerable uncertainty as to which model is best when performing model selection.  
4. The results of the outer loop of the nested cross validation is indicative of the ability of a model to generalize. 